# MCP + GPT‑5 Hands‑On

This notebook walks you through the **Model Context Protocol (MCP)** and **OpenAI GPT‑5** with a practical, step‑by‑step approach.
You'll:
- Understand the MCP client/server pattern
- Build a tiny MCP server (Python) and call a tool
- Connect an LLM workflow that calls **GPT‑5** via the **OpenAI Python SDK** (Responses API)
- Try structured outputs, function/tool calling, and safety tips

> References (keep handy):
- MCP overview & spec: modelcontextprotocol.io
- OpenAI API docs (GPT‑5, Responses API, Python SDK)


## 📦 Prerequisites
**You need:**
- Python 3.10+
- An OpenAI API key with access to GPT‑5 (`OPENAI_API_KEY`)
- (Optional) NodeJS if you want to explore MCP in Node later

**Create a virtual environment (optional):**
```bash
python -m venv .venv && source .venv/bin/activate  # Windows: .venv\Scripts\activate
```


## 🔧 Install Dependencies
We’ll use:
- `openai` — Official OpenAI Python SDK (v1+)
- `mcp` tooling — fast server helper from MCP Python SDK (`mcp`/`mcp[fast]` depending on packaging)
- `pydantic` — for structured outputs (optional but handy)


In [ ]:
# If running locally, uncomment to install
# %pip install --upgrade openai pydantic
# %pip install --upgrade "mcp" "mcp[fast]"  # or: %pip install modelcontextprotocol


In [ ]:
%pip install --upgrade openai pydantic

In [ ]:
%pip install --upgrade "mcp" "mcp[fast]"

## 🔐 Configure Environment
Set your API key securely. In a shell:
```bash
export OPENAI_API_KEY='sk-...'
```
Or within this notebook **for demo only** (avoid committing keys):

In [1]:
import os
# os.environ['OPENAI_API_KEY'] = 'sk-...'  # ⚠️ Demo only. Prefer environment variables.
assert 'OPENAI_API_KEY' in os.environ or True

## 🌐 Quick Intro: What is MCP?
MCP is a **client–server** protocol that standardizes how LLM apps connect to **tools, data sources, and workflows**. Think of it as a **USB‑C for AI apps** — build a server once and any MCP‑compatible client can use it.

**Core pieces:**
- **MCP Server**: wraps tools/data (e.g., a calculator, database, Jira, filesystem)
- **MCP Client/Host**: your app or IDE that talks to LLMs & routes tool calls
- **Transport**: stdio, HTTP, WebSockets, etc.


## 🧪 Lab 1 — Build a Tiny MCP Server (Python)
We'll expose one simple tool `add(a, b)` and one dataset `motd` using the fast MCP helper.

**Folder structure (suggested):**
```
mcp_demo/
  server.py
  client.py (optional)
```


In [36]:
%%writefile mcp_server.py

# mcp_server.py
import asyncio
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("DemoServer")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two integers."""
    return a + b

@mcp.tool()
def message_of_the_day() -> str:
    """Return a motivational message."""
    return "Stay curious. Build boldly."

if __name__ == "__main__":
    # Run using stdio transport
    mcp.run()

Overwriting mcp_server.py


In [ ]:
!python mcp_server.py

## 🧪 Lab 2 — Minimal MCP Client (Python)

Let's connect to our server and call the `add` tool. Different client helpers exist; here we illustrate a minimal stdio client to send a JSON‑RPC style request.

For production, prefer the official MCP client helpers when available.

In [42]:
%%writefile mcp_client.py

import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def main():
    params = StdioServerParameters(command="python", args=["mcp_server.py"])

    async with stdio_client(params) as (read_stream, write_stream):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()

            tools = await session.list_tools()
            # Each entry is (name, schema_dict)
            tool_names = [t[0] for t in tools]
            print("Available tools:", tool_names)

            # Call a tool
            motd = await session.call_tool("message_of_the_day", {})
            print("message_of_the_day() =", motd.structuredContent.get("result"))

            result = await session.call_tool("add", {"a": 10, "b": 5})
            print("add(10,5) =", result.structuredContent.get("result"))

asyncio.run(main())

Overwriting mcp_client.py


In [49]:
!python mcp_client.py

[10/31/25 22:36:27] INFO     Processing request of type            ]8;id=442060;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=135039;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             ListToolsRequest                                   
Available tools: ['meta', 'nextCursor', 'tools']
                    INFO     Processing request of type            ]8;id=250958;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=436207;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\
                             CallToolRequest                                    
message_of_the_day() = Stay curious. Build boldly.
                    INFO     Processing request of type            ]8;id=157347;file:///opt/anaconda3/lib/python3.11/site-packages/mcp/server/lowlevel/serv

In [ ]:
!pip show mcp

**Note:** 

Real clients manage sessions, schemas, discovery, and streaming.
See the MCP docs for full client helpers and message formats.


## 🤖 GPT‑5 with OpenAI Python SDK (Responses API)
We will use the **Responses API** for a unified interface (chat + tools + multimodal). Make sure your `openai` package is v1+.

**Basic text generation:**

In [6]:
# Basic GPT‑5 text generation using the Responses API
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

resp = client.responses.create(
    model="gpt-5" ,  # or the exact variant you have access to, e.g., "gpt-5-large"
    input="Explain the Model Context Protocol in one short paragraph for a banking audience."
)
print(resp.output_text)


The Model Context Protocol (MCP) is an open standard that lets AI assistants securely interact with your bank’s existing systems—databases, CRMs, payment rails—through governed connectors that expose only approved data and actions. Instead of copying data into the model, the assistant requests operations via MCP while your infrastructure enforces authentication, least-privilege access, redaction, and full audit logging, keeping sensitive information inside your environment. This enables use cases like KYC summaries, risk reports, or workflow initiation with human approval where needed. MCP standardizes tool access, reduces integration effort, and supports regulatory and data residency requirements.


### 🧱 Structured Outputs (JSON)
Ask GPT‑5 to return **well‑typed JSON** you can parse programmatically. Use instructions + `response_format="json_object"` or tool schemas when available.


In [4]:
from openai import OpenAI
from json import loads
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

resp = client.responses.create(
    model="gpt-5",
    input=(
        "Return a JSON object with keys: 'title' (string), 'summary' (string), "
        "and 'tags' (array of strings) about MCP in finance."
    ),
)

data = loads(resp.output_text)
data


{'title': 'Market Clearing Price (MCP) in Financial and Energy Markets',
 'summary': 'The Market Clearing Price (MCP) is the price at which aggregate supply equals aggregate demand, clearing the market without surplus or shortage. In financial exchanges and auctions, MCP is determined by matching buy and sell orders—often via call auctions or continuous order books—and sets the uniform transaction price for all matched volume at the clearing event. In power and energy markets, MCP typically refers to the uniform price in day-ahead or intraday auctions, distinct from locational marginal prices that incorporate transmission constraints. MCP underpins price discovery, allocation efficiency, and settlement, and is central to strategy design for bidders and market makers. Practitioners monitor MCP dynamics to manage execution risk, slippage, and volatility, and hedge exposure using futures, options, or bilateral contracts. Key considerations include auction design, tick size, market depth, 

In [ ]:
#pip install --upgrade openai

### 🛠️ Tool (Function) Calling Pattern
Let the model decide when to call your function to fetch external data. In Responses API, register **tools** with JSON schema and handle **tool calls**.


## 🛡️ Security & Safety Checklist
- **Prompt‑injection**: sanitize all model‑provided tool arguments; whitelist operations
- **AuthZ**: scope your MCP servers; avoid broad filesystem access in prod
- **Rate limits**: add caching and backoff around API calls
- **Privacy**: log minimally; scrub PII; encrypt at rest & in transit
- **Observability**: capture traces/metrics (latency, tool use, costs)


## 🧰 Troubleshooting
- `ModuleNotFoundError`: verify your venv and `pip show openai mcp`
- Tool call schema mismatches: ensure parameter names/types match your MCP tool
- Empty `output_text`: check rate limits or ensure your model name is correct and you have access
- For MCP transport issues: try `stdio` locally before websockets/HTTP
